This notebook is intended to look into the CO2 readings from the Mauna Loa Observatory, provided by the [Scripps CO2 Program](https://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo.html).

https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/daily/daily_in_situ_co2_mlo.csv

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## Acquisition

# TO DO: MAKE DECISION AND MOVE ON

OPTIONS:
- Include CSV
- Download directly:
  - Cut to today's snapshot
    - Would need to handle future header changes
  - Use entire file
    - My preferred - sort of - for general plots?

##### Using Data downloaded on 28JUN2022


In [3]:
#glancing at the CSV, the header starts on line 34 (only metadata above)
df = pd.read_csv("daily_in_situ_co2_mlo_28JUN2022.csv",header=33)

In [6]:
df.info() #23557 - grabs headers

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23557 entries, 0 to 23556
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   % Yr     23557 non-null  int64  
 1    Mn      23557 non-null  int64  
 2    Dy      23557 non-null  int64  
 3       CO2  23557 non-null  object 
 4    NB      23557 non-null  object 
 5    scale   23557 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.1+ MB


In [7]:
df2 = pd.read_csv("daily_in_situ_co2_mlo_28JUN2022.csv",comment="%")
df2.info() #23356 - no headers

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23556 entries, 0 to 23555
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   1958     23556 non-null  int64  
 1    01      23556 non-null  int64  
 2    01.1    23556 non-null  int64  
 3       NaN  23556 non-null  object 
 4     0      23556 non-null  object 
 5    12.0    23556 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.1+ MB


### NEED TO BE ABLE TO GRAB HEADERS AND SKIP THOSE COMMENTS


##### Using Data downloaded day of  **TIME PERMITTING**

Things to consider:
- Does/can the metadata change
  - May Change Header start line
  - Header line may still start with "% Yr" however metadata itself has that in it
    - So maybe use col 1 == "% Yr" and col 2 == "Mn"
    - % seems to indicate non data rows.  So maybe I can somehow use that to find last % line, which is header line
- Time split
  - Would need to not hard code any dates
    - All relative
  - Would need an appropriate split
    - T - X years
  - If seasonality chooses full years, would need to handle that as well
- Date
  - Would want to pull date (of download), but could just use last reading date

**Direct Link to csv:** https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/daily/daily_in_situ_co2_mlo.csv
